In [1]:
import os
import requests
from bs4 import BeautifulSoup

In [ ]:
# Scrape and save images in batches
def scrape_images(website_url, folder, batch_size=50):
    response = requests.get(website_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    img_tags = soup.find_all('img')
    os.makedirs(folder, exist_ok=True)

    for i, img in enumerate(img_tags[:batch_size]):  # Process only batch_size images at a time
        img_url = img.get('src')
        if img_url:
            # Handle relative URLs
            if img_url.startswith('//'):
                img_url = 'https:' + img_url
            elif img_url.startswith('/'):
                img_url = website_url.rstrip('/') + img_url
            
            try:
                # Download image
                response = requests.get(img_url, stream=True)
                if response.status_code == 200:
                    with open(os.path.join(folder, f'image_{i+1}.jpg'), 'wb') as file:
                        for chunk in response.iter_content(1024):
                            file.write(chunk)
                    print(f"Downloaded: {img_url}")
            except Exception as e:
                print(f"Error downloading {img_url}: {e}")

In [ ]:
# Example usage
scrape_images('https://example.com', './raw_images', batch_size=50)